In [27]:
# !pip install transformers --user
# !pip install pandas --user
import gc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Adafactor
from datetime import datetime
from pytz import timezone

In [28]:
def get_hour():
    tmz = timezone('America/Sao_Paulo')
    return datetime.utcnow().astimezone(tmz).strftime("%H:%M:%S")

In [29]:
import logging
logging.basicConfig(level=logging.ERROR)

In [30]:
def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [31]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [32]:
# !pip install wandb
import wandb

In [33]:
project_name = 'ptt5-sum400-8ephocs-adafactor-small'

In [34]:
device = 'cuda'

In [35]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.ementa = self.data.ementa
        self.inteiro_teor = self.data.inteiro_teor

    def __len__(self):
        return len(self.ementa)

    def __getitem__(self, index):
        inteiro_teor = str(self.inteiro_teor[index])
        inteiro_teor = ' '.join(inteiro_teor.split())

        ementa = str(self.ementa[index])
        ementa = ' '.join(ementa.split())

        source = self.tokenizer.batch_encode_plus([inteiro_teor], max_length=self.source_len, pad_to_max_length=True,return_tensors='pt', truncation=True)
        target = self.tokenizer.batch_encode_plus([ementa], max_length=self.summ_len, pad_to_max_length=True,return_tensors='pt', truncation=True)

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [36]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, decoder_input_ids=y_ids, lm_labels=lm_labels)
        loss = outputs[0]
        
        empty_cache()
        
        if _ % 10 == 0:
            wandb.log({"Loss do treinamento": loss.item()})

        if _ % 500 == 0:
            print(f'Época: {epoch}, Loss:  {loss.item()} ({get_hour()})')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        empty_cache()

In [37]:
import os
os.environ['WANDB_API_KEY'] = '99e2345c917d402940bb92ff1bee8818219b8241'

In [38]:
def init_wandb(project_name):
    wandb.init(project=project_name) 
    config = wandb.config
    config.TRAIN_BATCH_SIZE = 4
    config.VALID_BATCH_SIZE = 1
    config.TRAIN_EPOCHS = 8
    config.VAL_EPOCHS = 1
    config.MAX_LEN = 500
    config.SUMMARY_LEN = 100

    return config

In [39]:
def init_training_pipeline(config):
    print('Iniciando pipeline...\n\n')

    model_name = 'unicamp-dl/ptt5-small-portuguese-vocab'
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    empty_cache()
    
    train_dataset = pd.read_csv('../../data/train/train-400-2500.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    train_dataset = train_dataset[['ementa','inteiro_teor']]
#     train_dataset.inteiro_teor = 'summarize: ' + train_dataset.inteiro_teor
    print('Exemplo de textos:')
    print(train_dataset.head(), '\n\n')
    
    val_dataset = pd.read_csv('../../data/train/validate-400-2500.csv', encoding='utf-8', error_bad_lines=False, engine="python")
    val_dataset = val_dataset[['ementa','inteiro_teor']]
#     val_dataset.inteiro_teor = 'summarize: ' + val_dataset.inteiro_teor

    print(f'Dataset de treino: {train_dataset.shape}')
    print(f'Dataset de teste: {val_dataset.shape}')

    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    train_params = {'batch_size': config.TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
    val_params = {'batch_size': config.VALID_BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    
    print('Instanciando modelo...')
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    model = model.to(device)

    optimizer = Adafactor(
        model.parameters(),
        lr=1e-4,
        eps=(1e-40, 1e-4),
        clip_threshold=1.0,
        decay_rate=-0.8,
        beta1=None,
        weight_decay=0.0,
        relative_step=False,
        scale_parameter=False,
        warmup_init=False
    )

    empty_cache()

    wandb.watch(model, log="all")
    
    print('Inicializando Fine-Tuning utilizando o dataset de acórdãos...')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

        empty_cache()

    print('Treinamento concluído!\n\n')

    return tokenizer, model, val_loader

In [40]:
config = init_wandb(project_name)

wandb: Waiting for W&B process to finish, PID 16335
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20201204_051328-21u82bnc/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20201204_051328-21u82bnc/logs/debug-internal.log
wandb: Run summary:
wandb:                                                                 Loss do treinamento 0.6767126321792603
wandb:                                                                               _step 2160
wandb:                                                                            _runtime 25040
wandb:                                                                          _timestamp 1607083849
wandb: Run history:
wandb:   Loss do treinamento █▆▃▇▄▆▃▆▄▇▅▇▄▁▂▄▃▃▆▂▂▅▂▂▃▃▃▂▄▂▃▂▄▃▂▄▃▄▅▁
wandb:                 _step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:              _runtime ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:            _timestamp ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█

In [41]:
%%time
tokenizer, model, val_loader = init_training_pipeline(config)

Iniciando pipeline...


Exemplo de textos:
                                              ementa  \
0  agravo de instrumento. acidente do trabalho. r...   
1  agravo regimental. ação cautelar inominada. pe...   
2  embargos de declaração. vícios. inexistência.\...   
3  agravo de instrumento. tempestividade. orienta...   
4  agravo de instrumento. terceirização ilícita. ...   

                                        inteiro_teor  
0  acórdão\n\n6ª turma acv/rp \n\nvistos, relatad...  
1  a c ó r d ã o\n\n(ac. 1ª turma) gmwoc/db \n\nv...  
2  acórdão\n\n5ª turma emp/igr \n\nvistos, relata...  
3  a c ó r d ã o\n\n(6ª turma) gmacc\n\n/sc/jr/pv...  
4  acórdão\n\n6ª turma acv/acc \n\nvistos, relata...   


Dataset de treino: (10791, 2)
Dataset de teste: (2698, 2)
Instanciando modelo...
Inicializando Fine-Tuning utilizando o dataset de acórdãos...
Época: 0, Loss:  6.201450824737549 (09:11:00)


/home/labsuser/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/home/labsuser/.local/lib/python3.6/site-packages/transformers/modeling_t5.py:1128: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


Época: 0, Loss:  1.5637404918670654 (09:14:30)
Época: 0, Loss:  1.040911078453064 (09:18:04)
Época: 0, Loss:  0.9673894047737122 (09:21:35)
Época: 0, Loss:  1.0454820394515991 (09:25:07)
Época: 0, Loss:  0.5049256086349487 (09:28:38)
Época: 1, Loss:  0.643362820148468 (09:30:02)
Época: 1, Loss:  0.7287195324897766 (09:33:36)
Época: 1, Loss:  0.8226318955421448 (09:37:12)
Época: 1, Loss:  0.2226622998714447 (09:40:47)
Época: 1, Loss:  1.0666933059692383 (09:44:22)
Época: 1, Loss:  0.7958590984344482 (09:47:58)
Época: 2, Loss:  0.510918915271759 (09:49:24)
Época: 2, Loss:  0.3339237570762634 (09:53:01)
Época: 2, Loss:  0.8914083242416382 (09:56:37)
Época: 2, Loss:  0.29317525029182434 (10:00:12)
Época: 2, Loss:  0.49283236265182495 (10:03:48)
Época: 2, Loss:  0.7365700006484985 (10:07:22)
Época: 3, Loss:  0.2922365963459015 (10:08:49)
Época: 3, Loss:  0.3525083363056183 (10:12:23)
Época: 3, Loss:  0.8725384473800659 (10:15:59)
Época: 3, Loss:  0.7232889533042908 (10:19:34)
Época: 3, Loss

In [42]:
open("predictions.csv","w+")

<_io.TextIOWrapper name='predictions.csv' mode='w+' encoding='UTF-8'>

In [43]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    source_texts = []
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype=torch.long)
            y = data['target_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=250,
                num_beams=2,
                repetition_penalty=1.5,
                length_penalty=0.95,
                early_stopping=True
            )
            
            src_texts = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in ids]
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            if _ % 100 == 0 and _ > 0:
                print(f'Resumos: {_} gerados')
            
            source_texts.extend(src_texts)
            predictions.extend(preds)
            actuals.extend(target)

    return source_texts, predictions, actuals

In [44]:
def validade_and_save_predictions(val_epochs, tokenizer, model, val_loader):
    print('Gerando sumários utilizando o modelo no dataset de validação...')
    for epoch in range(val_epochs):
        source_texts, predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({ 'inteiro_teor': source_texts, 'ementa_original': actuals, 'resumo_gerado': predictions })
        final_df.to_csv('predictions.csv')
        print('CSV para análise gerado!')

In [45]:
%%time
validade_and_save_predictions(config.VAL_EPOCHS, tokenizer, model, val_loader)

Gerando sumários utilizando o modelo no dataset de validação...
Resumos: 100 gerados
Resumos: 200 gerados
Resumos: 300 gerados
Resumos: 400 gerados
Resumos: 500 gerados
Resumos: 600 gerados
Resumos: 700 gerados
Resumos: 800 gerados
Resumos: 900 gerados
Resumos: 1000 gerados
Resumos: 1100 gerados
Resumos: 1200 gerados
Resumos: 1300 gerados
Resumos: 1400 gerados
Resumos: 1500 gerados
Resumos: 1600 gerados
Resumos: 1700 gerados
Resumos: 1800 gerados
Resumos: 1900 gerados
Resumos: 2000 gerados
Resumos: 2100 gerados
Resumos: 2200 gerados
Resumos: 2300 gerados
Resumos: 2400 gerados
Resumos: 2500 gerados
Resumos: 2600 gerados
CSV para análise gerado!
CPU times: user 1h 33min 45s, sys: 1min 15s, total: 1h 35min 1s
Wall time: 56min 11s


In [46]:
generated_summaries = pd.read_csv('predictions.csv', encoding='utf-8', error_bad_lines=False, engine="python")
for index, row in generated_summaries[650:680].iterrows():
    print(f'Exemplo {index}', '\n')
    print('Inteiro teor:', row['inteiro_teor'], '\n\n')
    print('Ementa original:', row['ementa_original'], '\n\n')
    print('Sumário gerado:', row['resumo_gerado'], '\n\n')

Exemplo 650 

Inteiro teor: acórdão 8a turma dmc/al/dr/sm vistos, relatados e discutidos estes autos de agravo de instrumento em recurso de revista n o tst-airr-77340-78.2008.5.01.000, em que é agravante fundação petrobras de seguridade social - petros e são agravados josé carlos ferreira novo e petróleo brasileiro s.a. – petrobras. a vice-presidente do tribunal regional do trabalho da 1a região, pelo despacho de fls. 213/217, denegou seguimento ao recurso de revista da segunda reclamada, petros, por não vislumbrar violação dos dispositivos legais e constitucionais invocados, tampouco divergência jurisprudencial apta a ensejar a admissibilidade da revista. inconformada, a petros interpõe agravo de instrumento, às fls. 2/8, com a pretensão de desconstituir os fundamentos consignados na decisão denegatória do recurso de revista. foram apresentadas contraminuta ao agravo de instrumento e contrarrazões ao recurso de revista, em peça única, às fls. 222/242. desnecessária a remessa dos autos

In [47]:
# !pip install rouge-score tqdm --user
from tqdm import tqdm_notebook
# import time
from rouge_score import rouge_scorer, scoring
from typing import List, Dict

In [48]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [49]:
ROUGE_KEYS = ["rouge1", "rouge2", "rougeL"]
def calculate_rouge(output_lns: List[str], reference_lns: List[str]) -> Dict:
    scorer = rouge_scorer.RougeScorer(ROUGE_KEYS, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for reference_ln, output_ln in tqdm_notebook(zip(reference_lns, output_lns)):
        scores = scorer.score(reference_ln, output_ln)
        aggregator.add_scores(scores)

    result = aggregator.aggregate()
    return {k: v.mid.fmeasure for k, v in result.items()}

In [50]:
metrics = calculate_rouge(generated_summaries['ementa_original'], generated_summaries['resumo_gerado'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [51]:
metrics

{'rouge1': 0.7105489208075264,
 'rouge2': 0.6080287681763816,
 'rougeL': 0.666411140269434}

In [53]:
model_archive_name = 'ptt5-model'
print('Salvando modelo treinado...')
model.save_pretrained(model_archive_name)
print(f'Modelo salvo na pasta {model_archive_name}!\n')

Salvando modelo treinado...
Modelo salvo na pasta ptt5-model!

